In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [3]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [4]:
df = pd.read_csv('../datasets/mdlon.csv')

In [5]:
df.head()

,at1,at2,at3,at4,at5,at6,at7,at8,at9,at10,...,at492,at493,at494,at495,at496,at497,at498,at499,at500,T
0,485,477,537,479,452,471,491,476,475,473,...,481,477,485,511,485,481,479,475,496,-1
1,483,458,460,487,587,475,526,479,485,469,...,478,487,338,513,486,483,492,510,517,-1
2,487,542,499,468,448,471,442,478,480,477,...,481,492,650,506,501,480,489,499,498,-1
3,480,491,510,485,495,472,417,474,502,476,...,480,474,572,454,469,475,482,494,461,1
4,484,502,528,489,466,481,402,478,487,468,...,479,452,435,486,508,481,504,495,511,1


In [6]:
df.shape

(2000, 501)

In [7]:
bdf = BaseDatasetTransform(df,target='T')

In [8]:
df,categorical_features = bdf.fit_transform()


В колонке T нет пропущенных значений
-------------------------------------------
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index([], dtype='object')

Числовые колонки:
Index(['at1', 'at2', 'at3', 'at4', 'at5', 'at6', 'at7', 'at8', 'at9', 'at10',
       ...
       'at492', 'at493', 'at494', 'at495', 'at496', 'at497', 'at498', 'at499',
       'at500', 'T'],
      dtype='object', length=501)


In [9]:
X_train, X_test, y_train, y_test  = bdf.get_train_test_split()

Количество значений целевой переменной по категориям
T
-1    1000
 1    1000
Name: count, dtype: int64
Следует ли выполнить стратифицированное раздеение на обучающую и тестовую выборку? y/n


 n


Разделение датасета выполнено успешно


In [11]:

lgbm_model = lgb.LGBMClassifier(verbose=-1)


columns_to_delete = []  


model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    objective='binary',
    metric='binary_error'
)

In [12]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

ModelEvaluator(metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=[],
                                            columns_to_delete=[],
                                            model=LGBMClassifier(metric='binary_error',
                                                                 objective='binary',
                                                                 verbose=-1)))

In [13]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,Metric,Score
0,accuracy,0.758333
1,precision,0.758270
2,recall,0.758238
3,f1,0.758252
